In [128]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [129]:
drinks = pd.read_csv('/kaggle/input/starbucks-menu/starbucks_drinkMenu_expanded.csv')
drinks.head()

In [130]:
for col in drinks:
    print(col)

Remove spaces from the left

In [131]:
drinks.columns=drinks.columns.str.lstrip()

for col in drinks:
    print(col)

In [132]:
drinks.columns

Remove spaces from the right for certain columns as seen above

In [133]:
drinks.columns=drinks.columns.str.rstrip()

drinks.columns

Now we would like to check if there are any null values in the dataset

In [134]:
drinks.isna().any()

Null values exist in the column - Caffeine (mg)
Now let's check how many null values exist

In [135]:
drinks.isnull().sum()

There is only one null value, let's spot it!

In [136]:

drinks[drinks.isnull().any(axis=1)]

THe null value exists in the 158th row, lets fix it

We could not find the nutrition information that perfectly corresponds to the one in our initial dataset.

Referring to the link below we assumed the caffeine content of the drink is 125 (mg)

https://www.starbucks.com/menu/product/482/iced?parent=%2Fdrinks%2Fcold-coffees%2Ficed-coffees

In [137]:
new_drinks = drinks.fillna(125)
new_drinks.loc[[158]]

The null value has been fixed successfully

In [138]:
new_drinks.isna().any()

No more null values in any columns.

Now we need to check data types of each column

In [139]:
new_drinks.info()

We want the data type for 'Total Fat (g)', 'Vitamin A (% DV)', 'Vitamin C (% DV)', 'Calcium (% DV)', 'Iron (% DV)' and 'Caffeine (mg)' to be either integer-type or float-type, otherwise data plotting is not feasible.

While we supsect that there is a typo for 'Total Fat (g)', it seems that for the other aforementioned columns the system is interpreting the data-type as object because of the '%' sign.

First let's start with checking the unique values for 'Total Fat (g)'.

In [140]:
print(new_drinks['Total Fat (g)'].unique())

Because of the last value - 3 2, system identified the data type as object, let's fix the value

Let's check what the correct value should be

According to the Starbucks website the Total Fat should between 11 to 17 (g) depending on the size, so let's assume it's 16

In [141]:
new_drinks.loc[new_drinks['Total Fat (g)'] == '3 2']

In [142]:
new_drinks['Total Fat (g)'] = new_drinks['Total Fat (g)'].str.replace('3 2','16')
new_drinks.loc[[237]]

Replaced the total fat for index 237 to 16 - a proper integer

Now let's convert the values to float type

In [143]:
new_drinks['Total Fat (g)'] = new_drinks['Total Fat (g)'].astype(float)
print(new_drinks['Total Fat (g)'].dtypes)

Now, let's go over the same process for the below columns
'Vitamin A (% DV)', 'Vitamin C (% DV)', 'Calcium (% DV)', 'Iron (% DV)' and 'Caffeine (mg)

In [144]:
print(new_drinks["Vitamin A (% DV)"].unique())

For all columns in percent, we need to remove percent from the value

In [145]:
new_drinks['Vitamin A (% DV)'] = new_drinks['Vitamin A (% DV)'].str.replace('%', '')
new_drinks['Vitamin A (% DV)'] = new_drinks['Vitamin A (% DV)'].astype(int)
print(new_drinks['Vitamin A (% DV)'].dtypes)

In [146]:
new_drinks['Vitamin C (% DV)'] = new_drinks['Vitamin C (% DV)'].str.replace('%', '')
new_drinks['Vitamin C (% DV)'] = new_drinks['Vitamin C (% DV)'].astype(int)
print(new_drinks['Vitamin C (% DV)'].dtypes)

In [147]:
new_drinks['Calcium (% DV)'] = new_drinks['Calcium (% DV)'].str.replace('%', '')
new_drinks['Calcium (% DV)'] = new_drinks['Calcium (% DV)'].astype(int)
print(new_drinks['Calcium (% DV)'].dtypes)


In [148]:
print(new_drinks["Iron (% DV)"].unique())

Iron has both int and float values so converting the dtype to float64

In [181]:
new_drinks['Iron (% DV)'] = new_drinks['Iron (% DV)'].str.replace('%', '')
new_drinks['Iron (% DV)'] = new_drinks['Iron (% DV)'].astype(float)
print(new_drinks["Iron (% DV)"].dtypes)

Next up, caffiene

In [150]:
print(new_drinks["Caffeine (mg)"].unique())

We see 2 instances of Varies, which we need to replace with int value

Finding loc for 'varies'

In [151]:
new_drinks.loc[new_drinks["Caffeine (mg)"]== 'varies']

Since 'Tazo® Full-Leaf Tea Latte' is the only beverage with 'varies' for 'Caffeine (mg)', let's assume it's 50

In [152]:
new_drinks["Caffeine (mg)"] = new_drinks["Caffeine (mg)"].replace('varies','50')
print(new_drinks["Caffeine (mg)"].unique())

Replacing values for 'Varies'

In [153]:
new_drinks.loc[new_drinks["Caffeine (mg)"]== 'Varies']

We will assume the caffeine content for the same beverage category is constant.

(Tazo® Tea Drinks = 10, Shaken Iced Beverages = 20, Smoothies = 30)

In [154]:
new_drinks.iloc[102:106]=new_drinks.iloc[102:106].replace('Varies','10')
new_drinks.iloc[102:106]

In [155]:
new_drinks.iloc[167:172]=new_drinks.iloc[167:172].replace('Varies','20')
new_drinks.iloc[172]=new_drinks.iloc[172].replace('Varies','30')
new_drinks.iloc[167:173]

Checking unique values again to verify

In [156]:
print(new_drinks["Caffeine (mg)"].unique())

Changing columns data type to int

In [157]:
new_drinks["Caffeine (mg)"] = new_drinks["Caffeine (mg)"].astype(int)

In [158]:
print(new_drinks["Caffeine (mg)"].dtypes)

In [159]:
new_drinks.info()

Finally we have the dataset ready after cleaning null values and fixing data types

Now we can dive into the analysis and data plotting

In [160]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

In [161]:
new_drinks.Beverage_category.unique()

In [162]:

plt.figure(figsize=(10,6))
a=sns.countplot(x='Beverage_category',color='darkgreen',data=new_drinks)


xticks=plt.xticks(rotation=65,family='serif')
yticks=plt.yticks(family='serif')

plt.xlabel(new_drinks['Beverage_category'].all(),family='serif')
plt.ylabel('Count',family='serif')

a.spines['bottom'].set_color('gray')
a.spines['left'].set_color('gray')
sns.despine()

Here we can see number of items per drink type. Classic Espresso Drinks have the highest number of items followed by Tazo Tea Drinks and Signature Espresso Drinks

In [163]:
new_drinks.Beverage.unique()

In [164]:
px.density_heatmap(x="Beverage_category",y="Calories", data_frame = new_drinks, width=900)

Signature Espresso Drinks and Frappucino blended coffee have the highest number of high calorie beverages

We can see one drink under Signature Espresso Drinks with the highest calorie, let's find out what that is

In [165]:
new_drinks.loc[new_drinks["Calories"] >= 500]

White Chocolate Mocha (Without Whipped Cream) is the highest calorie item on the Starbucks menu with whopping 510 calories

Next up we analyze how are calories and sugar corelated

In [166]:
sns.scatterplot(x='Calories',y='Sugars (g)',data=new_drinks)
plt.title('Calories vs Sugar')

We can see Calories and sugar share a linear graph

In [167]:
sns.scatterplot(x='Calories',y='Protein (g)',data=new_drinks)


Now, we are going to devide this berverage into 2 groups. One is 'Hot Beverages' another is 'Cold Beverages'.

We will be observed the average of nutrients of each berverage in each group.

Before that, we are going to get rid of some nutrients to describe the only tidy data.

In [168]:
new_nutrients= new_drinks.drop(['Trans Fat (g)','Saturated Fat (g)','Sodium (mg)','Total Carbohydrates (g)',
                            'Cholesterol (mg)','Dietary Fibre (g)','Vitamin A (% DV)','Vitamin C (% DV)','Calcium (% DV)','Iron (% DV)'],axis=1)
new_nutrients.head()

In [169]:
Hot_drink = new_nutrients.drop(new_nutrients.index[154:]) # index starts from 0, so 154 is actually number 155 so 155 ~ at the end.
Hot_drink.head()

In [170]:
Hot_drink.iloc[-1] #check if it is the last Hot_drink

In [171]:
Cold_drink = new_nutrients.drop(new_nutrients.index[0:156]) # 0~155
Cold_drink.head()

In [172]:
Cold_drink.tail()

Now we'll be analyizing nutrients by hot drinks and cold drinks

In [173]:
calories=pd.DataFrame(new_drinks.groupby('Beverage_category')['Calories'].mean())

fig = go.Figure(data=[go.Bar(
    x=calories.index,
    y=calories['Calories']
)])
fig.update_xaxes(title='Type')
fig.update_yaxes(title='Avg Calories')
fig.show()

Here we can see that highest amount of calories is for Smoothies and the lowest is our plain old Coffee

Diving deep into individual beverages now

In [174]:
calories=pd.DataFrame(new_drinks.groupby('Beverage')['Calories'].mean())

fig = go.Figure(data=[go.Bar(
    x=calories.index,
    y=calories['Calories']
)])
fig.update_xaxes(title='Item')
fig.update_yaxes(title='Avg Calories')
fig.update_layout(width=900, height = 900)
fig.show()

As mentioned once above, White Chocolate Mocha has the highest calorie value among Hot drinks

Let's do the same for Total Fats



In [175]:
fat=pd.DataFrame(new_drinks.groupby('Beverage')['Total Fat (g)'].mean())

fig = go.Figure(data=[go.Bar(
    x=fat.index,
    y=fat['Total Fat (g)']
)])
fig.update_xaxes(title='Item')
fig.update_yaxes(title='Avg Fat')
fig.update_layout(width=900, height = 900)
fig.show()

Again we see White Chocolate Mocha has the highest amount of fat

Next up Sugar

In [176]:
sugar=pd.DataFrame(new_drinks.groupby('Beverage')['Sugars (g)'].mean())

fig = go.Figure(data=[go.Bar(
    x=sugar.index,
    y=sugar['Sugars (g)']
)])
fig.update_xaxes(title='Item')
fig.update_yaxes(title='Avg Sugar')
fig.update_layout(width=900, height = 900)
fig.show()

As we can see Java Chip (Without whipped cream) has the highest amount of Sugars

Moving on to Cold drinks now

In [177]:
calories=pd.DataFrame(Cold_drink.groupby('Beverage')['Calories'].mean())
        
fig = go.Figure(data=[go.Bar(
    x=calories.index,
    y=calories['Calories'],
)])
fig.update_layout(width=900,height=900)
fig.update_xaxes(title='Item')
fig.update_yaxes(title='Avg Calories')
fig.show()

As we can see Java Chip (Without whipped cream) has the highest amount of Calories among cold drinks

In [178]:
fat=pd.DataFrame(Cold_drink.groupby('Beverage')['Total Fat (g)'].mean())
        
fig = go.Figure(data=[go.Bar(
    x=fat.index,
    y=fat['Total Fat (g)'],
)])
fig.update_layout(width=900,height=900)
fig.update_xaxes(title='Item')
fig.update_yaxes(title='Avg Fat')
fig.show()

Again, Java Chip without whipped cream tops the chart for highest sugar content in cold drinks

Let's try protein now

In [179]:
protein = pd.DataFrame(Cold_drink.groupby('Beverage')['Protein (g)'].mean())

fig = go.Figure(data = [go.Bar(
    x = protein.index,
    y = protein['Protein (g)']
)])

fig.update_layout(width = 900, height = 900)
fig.update_xaxes(title = 'Item')
fig.update_yaxes(title = 'Avg Protein')
fig.show()

We have a new winner - Banana Chocolate Smoothie has the highest Protein Content

With above analysis we can confirm, irrespective of hot or cold drink, Java Chip (without whipped cream) has the highest amount of sugar. And even in terms of calories, Java Chip w/o whipped cream has the highest in cold drinks and in terms of hot drinks java chip w/o whipped cream is a close second to White Chocolate Mocha